<a href="https://colab.research.google.com/github/NateshReddy/PCB-Failure-Analyser-playground/blob/main/pcb_try1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.chdir("/content/drive/MyDrive/Project/failure_analysis/notebooks")

In [ ]:
#import 
import shutil
import cv2
import numpy as np
import json
import glob
import pandas as pd
import zipfile

In [ ]:
# to create directories from storing images
'''os.mkdir("6_difference_img")
os.mkdir("6_extracted_defects")
os.mkdir("6_hold_out")'''

In [ ]:
image_path = 'DeepPCB/PCBData/'
temp_pattern = '/*temp.jpg'
test_pattern = '/*test.jpg'

temps_and_tests = {}

for folder in os.listdir(image_path):
    full_path = image_path + folder + '/' + folder.replace('group', '')
    temps = glob.glob(full_path + '/' + temp_pattern)
    tests = glob.glob(full_path + '/' + test_pattern)
    inner = {}
    inner['temps'] = temps
    inner['tests'] = tests
    temps_and_tests[folder] = inner

In [ ]:
len(temps_and_tests['group00041']['temps'])

221

In [ ]:
def subtract_images(image_path_1, image_path_2, write_path, choice):
    image1 = cv2.imread(image_path_1)
    image2 = cv2.imread(image_path_2)
    difference = cv2.subtract(image1, image2)
    Conv_hsv_Gray = cv2.cvtColor(difference, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(Conv_hsv_Gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    difference[mask != 255] = [0, 0, 255]
    image1[mask != 255] = [0, 0, 255]
    image2[mask != 255] = [0, 0, 255]
    if choice == 1:
      cv2.imwrite(write_path, image1)
    elif choice == 2:
      cv2.imwrite(write_path, image2)

In [ ]:
#!rm 6_difference_img/*

In [ ]:
split_percentage = round(len(temps_and_tests['group00041']['temps']) * 0.2)
last = 177
for index, whatever in enumerate(temps_and_tests['group00041']['temps']):
  #print(index)
  root_img_name = temps_and_tests['group00041']['temps'][index].replace('_temp.jpg', '')
  temp_file_path = root_img_name  + '_temp.jpg'
  test_file_path = root_img_name + '_test.jpg'
  write_path_b = "6_difference_img/diff_image_" + str(index) + ".png"
  write_path_w = "6_difference_img/diff_image_" + str(last + index) + ".png"
  if(index < split_percentage):
    print(index)
    '''for file in [temp_file_path]:
      shutil.copy(file, "6_hold_out/")
    for file in [test_file_path]:
      shutil.copy(file, "6_hold_out/")'''
  else:
    subtract_images(temp_file_path, test_file_path, write_path_b, 1) #black defects
    subtract_images(test_file_path, temp_file_path, write_path_w, 2) #white defects

In [ ]:
all_differenced_images = os.listdir('6_difference_img')

In [ ]:
len(all_differenced_images)

354

In [ ]:
def extract_contours_from_image(image_path, write_path, hsv_lower, hsv_upper):
    image = cv2.imread(image_path)
    original = image.copy()
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv_lower = np.array(hsv_lower)
    hsv_upper = np.array(hsv_upper)
    mask = cv2.inRange(hsv, hsv_lower, hsv_upper) #if red pixel, then 255 otherwise 0
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
    close = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel, iterations=1)
    cnts, _ = cv2.findContours(close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    
    offset = 20
    ROI_number = 0
    if cnts!=None:
      for c in cnts:
          x, y, w, h = cv2.boundingRect(c)
          cv2.rectangle(image, (x - offset, y - offset), (x + w + offset, y + h + offset), (36, 255, 12), 2)
          ROI = original[y - offset:y + h + offset, x - offset:x + w + offset]
          try:
              cv2.imwrite(write_path + 'contour_{}.png'.format(ROI_number), ROI)
          except:
              print("skipping image " + image_path + "for counter" + str(ROI_number))
          ROI_number += 1
    else:
      print("No counters found at " + image_path)
    

In [ ]:
for index, filename in enumerate(all_differenced_images):
    os.mkdir("6_extracted_defects/extracts_" + str(index))
    image_path = "6_difference_img/" + filename
    this_write_path = "6_extracted_defects/extracts_" + str(index) + "/"
    hsv_lower = [0,150,50], #for red colour
    hsv_upper = [10,255,255]
    extract_contours_from_image(image_path, this_write_path, hsv_lower, hsv_upper)

skipping image 6_difference_img/diff_image_222.pngfor counter9
skipping image 6_difference_img/diff_image_222.pngfor counter10
skipping image 6_difference_img/diff_image_222.pngfor counter11
skipping image 6_difference_img/diff_image_224.pngfor counter4
skipping image 6_difference_img/diff_image_224.pngfor counter5
skipping image 6_difference_img/diff_image_224.pngfor counter6
skipping image 6_difference_img/diff_image_50.pngfor counter0
skipping image 6_difference_img/diff_image_230.pngfor counter3
skipping image 6_difference_img/diff_image_54.pngfor counter4
skipping image 6_difference_img/diff_image_54.pngfor counter7
skipping image 6_difference_img/diff_image_54.pngfor counter17
skipping image 6_difference_img/diff_image_54.pngfor counter18
skipping image 6_difference_img/diff_image_54.pngfor counter19
skipping image 6_difference_img/diff_image_54.pngfor counter20
skipping image 6_difference_img/diff_image_54.pngfor counter21
skipping image 6_difference_img/diff_image_54.pngfor cou

In [ ]:
#!rm -r 6_extracted_defects/*

In [ ]:
check_all_extracted_folder = os.listdir('6_extracted_defects')
len(check_all_extracted_folder)

354

In [ ]:
os.mkdir("6_all_extracts")

In [ ]:
cnt=0
for dirpath, dirs, files in os.walk("6_extracted_defects"):
    for index, filename in enumerate(files):
        if filename.endswith(".png"):
            cnt += 1
            file_paths = [dirpath + "/" + filename]
            target_directory = "6_all_extracts/extract_" + str(cnt) + ".png"
            for file in file_paths:
              shutil.move(file, target_directory)

In [ ]:
check_all_extracted = os.listdir('6_all_extracts')
len(check_all_extracted)

2980

In [ ]:
!zip -r /6_extracted_defects.zip 6_all_extracts/

  adding: 6_all_extracts/ (stored 0%)
  adding: 6_all_extracts/extract_1.png (stored 0%)
  adding: 6_all_extracts/extract_2.png (deflated 3%)
  adding: 6_all_extracts/extract_3.png (stored 0%)
  adding: 6_all_extracts/extract_4.png (deflated 5%)
  adding: 6_all_extracts/extract_5.png (deflated 7%)
  adding: 6_all_extracts/extract_6.png (deflated 7%)
  adding: 6_all_extracts/extract_7.png (deflated 6%)
  adding: 6_all_extracts/extract_8.png (deflated 31%)
  adding: 6_all_extracts/extract_9.png (deflated 22%)
  adding: 6_all_extracts/extract_10.png (stored 0%)
  adding: 6_all_extracts/extract_11.png (deflated 4%)
  adding: 6_all_extracts/extract_12.png (deflated 12%)
  adding: 6_all_extracts/extract_13.png (deflated 6%)
  adding: 6_all_extracts/extract_14.png (deflated 19%)
  adding: 6_all_extracts/extract_15.png (deflated 3%)
  adding: 6_all_extracts/extract_16.png (deflated 4%)
  adding: 6_all_extracts/extract_17.png (deflated 14%)
  adding: 6_all_extracts/extract_18.png (deflated 8%)


In [ ]:
file_names_and_sizes = {}
file_names = os.listdir('6_all_extracts/')

for index, file in enumerate(file_names):
  file_names_and_sizes[file_names[index]] = os.path.getsize('6_all_extracts/' + file)

In [ ]:
file_names_and_sizes = {k: v for k, v in file_names_and_sizes.items() if v != 0}

In [ ]:
with open('6_extracted_failures.json', 'w') as outfile:
    json.dump(file_names_and_sizes, outfile)